In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import tqdm
import datetime


In [2]:
# read new_comprehensive_temp.csv
df = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\old_intermediate\comprehensive.csv')
# read IPC_new data, market_match data and precipitation data
IPC_new = gpd.read_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\(2) prepare for the new dataset\intermediate\IPC_new.geojson')
precipitation = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\old_intermediate\precipitation_sum.csv')

In [3]:
# does date,lat,lon form a unique key?
df[['date','lat','lon']].duplicated().sum()
# keep the first record of each date,lat,lon
df = df.drop_duplicates(subset=['date','lat','lon'],keep='first')

In [4]:
# keep only unique geometry, lat and lon
df_new = df.drop_duplicates(subset=['geometry','lat','lon'],keep='first')
df_new = df_new.reset_index(drop=True)
# keep only geometry, lat and lon
df_new = df_new[['geometry','lat','lon']]
#save df_new as geometry.csv
df_new.to_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\geometry.csv',index=False)

In [5]:
#see if df is unique on date,lat,lon
df[['date','lat','lon']].duplicated().sum()

0

In [6]:
# drop geometry
df = df.drop(columns=['geometry'])

In [7]:
df

,date,event_count_battles,event_count_explosions,event_count_violence,fatalities_battles,fatalities_explosions,fatalities_violence,GOSIF_GPP,rainfall_chirps,GOSIF_GPP_SD,elevation,soil,price,lon,lat,temperature_2m_mean,temperature_2m_mean_sd,shortwave_radiation_sum,shortwave_radiation_sum_sd
0,2017-01-01,1.0,0.0,0.0,5.0,0.0,0.0,22458.920383,109.417561,1198.248826,NaN,NaN,NaN,14.070903,-5.533893,26.122581,0.804450,NaN,NaN
1,2017-01-01,1.0,0.0,0.0,5.0,0.0,0.0,22200.090910,111.121122,1175.474359,NaN,NaN,NaN,14.068317,-5.493558,25.922581,0.804450,17.102903,3.211451
2,2017-01-01,1.0,0.0,0.0,5.0,0.0,0.0,22218.324112,138.828427,1342.802136,NaN,NaN,NaN,14.314904,-5.289988,26.306452,0.817286,17.347742,3.216938
3,2017-01-01,2.0,0.0,0.0,4.0,0.0,0.0,4970.260394,0.229789,1425.793103,NaN,NaN,NaN,15.778695,6.765411,25.380645,1.135318,22.995806,1.004940
4,2017-01-01,1.0,0.0,0.0,3.0,0.0,0.0,4307.993754,0.134368,1467.055556,NaN,NaN,NaN,16.437358,7.248169,26.880645,1.486252,22.840968,0.772474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40876,2022-12-01,0.0,1.0,0.0,0.0,3.0,0.0,784.389728,10.718039,1692.512253,NaN,NaN,NaN,71.456211,37.037830,-10.670968,3.213741,10.735161,1.558302
40877,2022-12-01,1.0,0.0,0.0,0.0,0.0,0.0,1252.600815,13.004443,1651.941176,NaN,NaN,NaN,71.487215,35.171917,6.951613,1.597054,10.958387,1.877639
40878,2022-12-01,1.0,1.0,1.0,1.0,0.0,1.0,1416.164120,9.017693,1422.424242,NaN,NaN,NaN,71.504960,34.752987,9.309677,1.784630,11.210000,1.876703
40879,2022-12-01,1.0,1.0,1.0,1.0,0.0,1.0,1467.059042,9.080978,1422.424242,NaN,NaN,NaN,71.504927,34.753595,9.309677,1.784630,11.210000,1.876703


In [8]:
# convert date to datetime
df['date'] = pd.to_datetime(df['date'])
# convert date in precipation to datetime
precipitation['date'] = pd.to_datetime(precipitation['date'])

In [9]:
# see if date,lat,lon in precipitation is unique
precipitation[['date','lat','lon']].duplicated().sum()
# drop duplicates in precipitation
precipitation = precipitation.drop_duplicates(subset=['date','lat','lon'],keep='first')

In [10]:
# merge df and precipitation, add a _merge column
df = df.merge(precipitation,how='left',on=['date','lat','lon'],indicator=True)

In [11]:
# check _merge column
df['_merge'].value_counts()


_merge
both          38058
left_only       718
right_only        0
Name: count, dtype: int64

In [12]:
# drop _merge column
df = df.drop(columns=['_merge'])

In [13]:
location = df[['date','lat','lon']]
# drop duplicates in location
location = location.drop_duplicates(subset=['date','lat','lon'],keep='first')

In [14]:
location

,date,lat,lon
0,2017-01-01,-5.533893,14.070903
1,2017-01-01,-5.493558,14.068317
2,2017-01-01,-5.289988,14.314904
3,2017-01-01,6.765411,15.778695
4,2017-01-01,7.248169,16.437358
...,...,...,...
38771,2022-12-01,37.037830,71.456211
38772,2022-12-01,35.171917,71.487215
38773,2022-12-01,34.752987,71.504960
38774,2022-12-01,34.753595,71.504927


In [15]:
# import FAO_price data xlsx file
FAO_price = pd.read_excel(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\0.external_data\FAO_price_data\FAO_DATA_FOR_MIGUEL.xlsx')

# keep only date, lat, lon, price and indexReferencePrice
FAO_price = FAO_price[['date', 'lat', 'long', 'price', 'indexReferencePrice']]
# new price index is price/indexReferencePrice*100
FAO_price['price_index'] = FAO_price['price']/FAO_price['indexReferencePrice']*100

# average on each date and location
FAO_price = FAO_price.groupby(['date', 'lat', 'long']).mean().reset_index()
#drop indexReferencePrice and price
FAO_price = FAO_price.drop(columns=['indexReferencePrice', 'price'])
import pandas as pd
from haversine import haversine, Unit
from sklearn.neighbors import BallTree


# unique all dates from location
dates = location['date'].unique()

# create a empty dataframe to store the data:date	lat	lon	nearest_neighbor_distance	market_lat	market_lon	price_index
market_match = pd.DataFrame(columns=['area_id','date', 'lat', 'lon', 'nearest_neighbor_distance', 'market_lat', 'market_lon', 'price_index'])

def to_radians(degrees):
    return degrees * (np.pi / 180)

# loop over all dates
for date in tqdm.tqdm(dates):
    # filter location by selecting the first date
    location_temp = location[location['date'] == date]
    # filter FAO_price by selecting the first date
    FAO_price_temp = FAO_price[FAO_price['date'] == date]
    # Function to convert degrees to radians
    # Convert latitude and longitude to radians
    coords1_rad = location_temp[['lat', 'lon']].apply(to_radians)
    coords2_rad = FAO_price_temp[['lat', 'long']].apply(to_radians)

    # Create a BallTree
    tree = BallTree(coords2_rad, metric='haversine')

    # Query the BallTree to find the index of the nearest neighbor in set2 for each point in set1
    distances, indices = tree.query(coords1_rad, k=1)

    # Add the nearest neighbor information to df1
    location_temp.loc[:, 'nearest_neighbor_index'] = indices.flatten()
    location_temp.loc[:, 'nearest_neighbor_distance'] = distances.flatten() * 6371  # Multiply by Earth's radius to get distance in km

    #reset index and add a new column index
    FAO_price_temp = FAO_price_temp.reset_index(drop=True)
    FAO_price_temp['index'] = FAO_price_temp.index

    # Merge the nearest neighbors from df2 into df1
    result = pd.merge(location_temp, FAO_price_temp, left_on='nearest_neighbor_index', right_on='index', suffixes=('_df1', '_df2'))
    # rename date_df1 to date, lat_df1 to lat, lat_df2 to market_lat, long to market_lon, drop nearest_neighbor_index,date_df2,index
    result = result.rename(columns={'date_df1': 'date', 'lat_df1': 'lat', 'lat_df2': 'market_lat', 'long': 'market_lon'})
    result = result.drop(columns=['nearest_neighbor_index', 'date_df2', 'index'])
    #concat result with market_match
    market_match = pd.concat([market_match, result], axis=0)

  0%|          | 0/72 [00:00<?, ?it/s]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_54220\4248554333.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_temp.loc[:, 'nearest_neighbor_index'] = indices.flatten()
C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_54220\4248554333.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_temp.loc[:, 'nearest_neighbor_distance'] = distances.flatten() * 6371  # Multiply by Earth's radius to get distance in km
C:\Users\WeilunShi\AppData\Local\Temp\ip

In [16]:
# drop area_id
market_match = market_match.drop(columns=['area_id'])

In [17]:
# see if market_match is unique on date,lat,lon
market_match[['date','lat','lon']].duplicated().sum()

0

In [18]:
# merge market_match with df, add indicator column
df = df.merge(market_match,how='left',on=['date','lat','lon'],indicator=True)

In [19]:
# drop price
df = df.drop(columns=['price'])

In [20]:
# see _merge
df['_merge'].value_counts()

_merge
both          38776
left_only         0
right_only        0
Name: count, dtype: int64

In [21]:
# drop _merge
df = df.drop(columns=['_merge'])

In [22]:
# read geojson data


In [23]:
# add the centroid to IPC_new
IPC_new['centroid'] = IPC_new.centroid

C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_54220\2373558919.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  IPC_new['centroid'] = IPC_new.centroid


In [24]:
# add lat and lon to IPC_new
IPC_new['lat'] = IPC_new.centroid.y
IPC_new['lon'] = IPC_new.centroid.x

C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_54220\1115186264.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  IPC_new['lat'] = IPC_new.centroid.y
C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_54220\1115186264.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  IPC_new['lon'] = IPC_new.centroid.x


In [25]:
IPC_new = IPC_new.drop(columns=['centroid'])

In [26]:
# convert date to datetime
IPC_new['date'] = pd.to_datetime(IPC_new['date'])

In [27]:
# drop geometry, convert IPC_new to dataframe
IPC_new = IPC_new.drop(columns=['geometry'])
IPC_new = pd.DataFrame(IPC_new)

In [28]:
# see if date,lat,lon is unique
IPC_new[['date','lat','lon']].duplicated().sum()

93

In [29]:
# drop duplicates
IPC_new = IPC_new.drop_duplicates(subset=['date','lat','lon'],keep='first')

In [30]:
location

,date,lat,lon
0,2017-01-01,-5.533893,14.070903
1,2017-01-01,-5.493558,14.068317
2,2017-01-01,-5.289988,14.314904
3,2017-01-01,6.765411,15.778695
4,2017-01-01,7.248169,16.437358
...,...,...,...
38771,2022-12-01,37.037830,71.456211
38772,2022-12-01,35.171917,71.487215
38773,2022-12-01,34.752987,71.504960
38774,2022-12-01,34.753595,71.504927


In [31]:
IPC_new

,estimated_population,date,title,overall_phase,country,phase3_worse_population,phase3_worse_percentage,phase1_population,phase1_percent,phase2_population,...,phase4_percent,phase5_population,phase5_percent,phase3_worse_percentage_manual,phase3_plus_phase4,phase2_worse_percentage_manual,phase2_plus_phase3,phase2_plus_phase3_plus_phase4,lat,lon
0,906391,2017-04-01,Dépressions de l`est,3,BI,271918,0.30,271917,0.30,362556,...,0.05,0.0,0.0,0.30,0.30,0.70,0.65,0.70,-3.654396,30.340291
1,1772906,2017-04-01,Plateaux secs de l`est,3,BI,709163,0.40,354581,0.20,709162,...,0.10,0.0,0.0,0.40,0.40,0.80,0.70,0.80,-3.227121,30.281089
2,238111,2017-04-01,Buragane,3,BI,47623,0.20,107150,0.45,83339,...,0.05,0.0,0.0,0.20,0.20,0.55,0.50,0.55,-4.236441,29.823939
3,1171450,2017-04-01,Crête congo nil,2,BI,222575,0.19,410008,0.35,538867,...,0.05,0.0,0.0,0.19,0.19,0.65,0.60,0.65,-3.410699,29.406879
4,651193,2017-04-01,Dépressions du nord,4,BI,293037,0.45,162798,0.25,195358,...,0.20,0.0,0.0,0.45,0.45,0.75,0.55,0.75,-2.511128,30.218558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9474,9604,2023-01-01,Zeylac Urban,2,SO,0,0.00,7680,0.80,1440,...,0.00,0.0,0.0,0.05,0.05,0.20,0.20,0.20,11.352035,43.472607
9475,130615,2023-02-01,Ocotepeque,2,LAC,0,0.00,96655,0.74,19592,...,0.01,0.0,0.0,0.11,0.11,0.26,0.25,0.26,14.466401,-89.054836
9476,12705,2023-02-01,Cayaguanca,2,LAC,0,0.00,9529,0.75,2033,...,0.01,0.0,0.0,0.09,0.09,0.25,0.24,0.25,14.262648,-89.104538
9477,246803,2023-02-01,Chorti,3,LAC,0,0.00,91317,0.37,101189,...,0.02,0.0,0.0,0.22,0.22,0.63,0.61,0.63,14.723058,-89.309428


In [32]:
location_2 = IPC_new[['date','lat','lon']]

In [33]:
from haversine import haversine, Unit
from sklearn.neighbors import BallTree


# unique all dates from location
dates = location['date'].unique()

# create a empty dataframe to store the data:date	lat	lon	nearest_neighbor_distance	market_lat	market_lon	price_index
market_match = pd.DataFrame(columns=['date', 'lat', 'lon', 'market_lat', 'market_lon'])

def to_radians(degrees):
    return degrees * (np.pi / 180)

# loop over all dates
for date in tqdm.tqdm(dates):
    # filter location by selecting the first date
    location_temp = location[location['date'] == date]
    # filter FAO_price by selecting the first date
    FAO_price_temp = location_2[location_2['date'] == date]
    # Function to convert degrees to radians
    # Convert latitude and longitude to radians
    coords1_rad = location_temp[['lat', 'lon']].apply(to_radians)
    coords2_rad = FAO_price_temp[['lat', 'lon']].apply(to_radians)

    # Create a BallTree
    if len(coords2_rad) > 0:
        tree = BallTree(coords2_rad, metric='haversine')

    # Query the BallTree to find the index of the nearest neighbor in set2 for each point in set1
    distances, indices = tree.query(coords1_rad, k=1)

    # Add the nearest neighbor information to df1
    location_temp.loc[:, 'nearest_neighbor_index'] = indices.flatten()
    location_temp.loc[:, 'nearest_neighbor_distance'] = distances.flatten() * 6371  # Multiply by Earth's radius to get distance in km

    #reset index and add a new column index
    FAO_price_temp = FAO_price_temp.reset_index(drop=True)
    FAO_price_temp['index'] = FAO_price_temp.index
    
    #reset index and add a new column index
    location_temp = location_temp.reset_index(drop=True)
    location_temp['index'] = location_temp.index

    # Merge the nearest neighbors from df2 into df1
    result = pd.merge(location_temp, FAO_price_temp, left_on='nearest_neighbor_index', right_on='index', suffixes=('_df1', '_df2'))
    # rename date_df1 to date, lat_df1 to lat, lat_df2 to market_lat, long to market_lon, drop nearest_neighbor_index,date_df2,index
    #concat result with market_match
    market_match = pd.concat([market_match, result], axis=0)
# drop date
market_match = market_match.drop(columns=['date','lat','lon','market_lat','market_lon','nearest_neighbor_index','index_df1','date_df2','index_df2'])
market_match
# rename date_df1 to date, lat_df1 to lat, lon_df1 to lon, lat_df2 to market_lat, lon_df2 to market_lon
market_match = market_match.rename(columns={'date_df1': 'date', 'lat_df1': 'lat', 'lon_df1': 'lon', 'lat_df2': 'market_lat_2', 'lon_df2': 'market_lon_2'})

  0%|          | 0/72 [00:00<?, ?it/s]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_54220\2325022024.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_temp.loc[:, 'nearest_neighbor_index'] = indices.flatten()
C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_54220\2325022024.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_temp.loc[:, 'nearest_neighbor_distance'] = distances.flatten() * 6371  # Multiply by Earth's radius to get distance in km
C:\Users\WeilunShi\AppData\Local\Temp\ip

In [34]:
# see date lat lon is unique
market_match[['date','lat','lon']].duplicated().sum()

0

In [35]:
# filter nearest_neighbor_distance < 200
market_match = market_match[market_match['nearest_neighbor_distance'] < 200]

In [36]:
# merge market_match with df, add indicator column
df = df.merge(market_match,how='left',on=['date','lat','lon'],indicator=True)

In [37]:
#describe _merge
df['_merge'].value_counts()

_merge
left_only     32807
both           5969
right_only        0
Name: count, dtype: int64

In [38]:
# drop _merge, nearest_neighbor_distance_y
df = df.drop(columns=['_merge','nearest_neighbor_distance_y'])

In [39]:
# see if date, lat, lon is unique in IPC_new
IPC_new[['date','lat','lon']].duplicated().sum()

0

In [40]:
# merge IPC_new with df, add indicator column, lef on date, market_lat_2, market_lon_2, right on date, lat, lon
df = df.merge(IPC_new,how='left',left_on=['date','market_lat_2','market_lon_2'],right_on=['date','lat','lon'],indicator=True)

In [41]:
# describe _merge
df['_merge'].value_counts()

_merge
left_only     32807
both           5969
right_only        0
Name: count, dtype: int64

In [42]:
# drop _merge, market_lat_2, market_lon_2
df = df.drop(columns=['_merge','market_lat_2','market_lon_2'])

In [43]:
# drop lat_y, lon_y
df = df.drop(columns=['lat_y','lon_y'])

In [44]:
# describe include all, transpose
df.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
date,38776,NaN,NaN,NaN,2020-01-03 08:26:23.494945536,2017-01-01 00:00:00,2018-07-01 00:00:00,2020-02-01 00:00:00,2021-07-01 00:00:00,2022-12-01 00:00:00,NaN
event_count_battles,38776.0,NaN,NaN,NaN,3.26341,0.0,0.0,1.0,3.0,141.0,7.826628
event_count_explosions,38776.0,NaN,NaN,NaN,1.633691,0.0,0.0,0.0,1.0,99.0,4.966043
event_count_violence,38776.0,NaN,NaN,NaN,1.513204,0.0,0.0,1.0,2.0,117.0,3.430215
fatalities_battles,38776.0,NaN,NaN,NaN,10.42075,0.0,0.0,0.0,6.0,897.0,34.749177
fatalities_explosions,38776.0,NaN,NaN,NaN,3.541495,0.0,0.0,0.0,0.0,624.0,15.813038
fatalities_violence,38776.0,NaN,NaN,NaN,2.370074,0.0,0.0,0.0,1.0,380.0,11.403577
GOSIF_GPP,38213.0,NaN,NaN,NaN,10262.171739,98.071429,2821.699279,6663.761751,17890.974925,33774.707702,8828.875303
rainfall_chirps,36088.0,NaN,NaN,NaN,66.549814,0.000041,6.676779,29.98975,106.595002,1221.385403,81.33474
GOSIF_GPP_SD,36003.0,NaN,NaN,NaN,1324.859972,612.0,975.5,1160.618182,1406.27838,6544.823529,575.523248


In [45]:
#export df to csv
df.to_csv('new_merge.csv',index=False)